# Reading histograms

For the new python-based frameworks, another common task would be to translate
histogram in the [`scikit-hep.hist`](https://hist.readthedocs.io/en/latest/)
package into the HEPData format. The functions in the `hepdata_lib` will help
you with that, and this notebook will demonstrate how to do that.

As explained in the [Getting started notebook](Getting_started.ipynb), a
`Submission` needs to exist or be created. Here, we'll just create one without
any additional information:

In [1]:
from hepdata_lib import Submission

submission = Submission()


Welcome to JupyROOT 6.28/04


The common use-case `scikit-hep` histograms is to allow for after-the-fact
slicing and grouping from a main histogram. Let us first generate a fake
histogram that may appear in common histograms, as well as a common slicing
routine

In [2]:
import hist
import numpy as np

rng = np.random.default_rng(seed=123_456_789)

h = hist.Hist(
    hist.axis.StrCategory(["data", "QCD", "ttbar"], name="dataset"),
    hist.axis.IntCategory([-1, 0, 4, 5], name="flavor"),
    hist.axis.Regular(60, -3, +3, name="eta"),
    hist.axis.Regular(20, 0, 500, name="pt"),
    storage=hist.storage.Weight(),
)

h.fill(  ## For mock data
    dataset="data",
    flavor=-1,
    eta=rng.normal(0, 2.0, size=123_456),
    pt=rng.exponential(100, size=123_456),
)
h.fill(  ## For Mock QCD
    dataset="QCD",
    flavor=rng.choice([0, 4, 5], size=1_000_000, p=[0.8, 0.15, 0.05]),
    eta=rng.normal(0.0, 2.0, size=1_000_000),
    pt=rng.exponential(100, size=1_000_000),
    weight=0.123456 * 2 * rng.random(size=1_000_000),
)
h.fill(  ## For mock ttbar
    dataset="ttbar",
    flavor=rng.choice([0, 4, 5], size=1_000_000, p=[0.45, 0.1, 0.45]),
    eta=rng.normal(0.0, 1.5, size=1_000_000),
    pt=rng.exponential(200, size=1_000_000),
    weight=0.01 * 2 * rng.random(size=1_000_000),
)


Hist(
  StrCategory(['data', 'QCD', 'ttbar'], name='dataset'),
  IntCategory([-1, 0, 4, 5], name='flavor'),
  Regular(60, -3, 3, name='eta'),
  Regular(20, 0, 500, name='pt'),
  storage=Weight()) # Sum: WeightedSum(value=221221, variance=123802) (WeightedSum(value=256973, variance=143935) with flow)

## Example of manual processing to 1D array

Let use create a simple slicing routine to get the various histograms of
interest, then use the most versatile function, the
`hepdata_lib.hist_utils.read_hist` method, to create arrays that will be
compatible with variable creation.

In [3]:
from hepdata_lib.hist_utils import read_hist

data_hist = h[dict(dataset='data', flavor=sum, eta=sum)]
fqcd_hist = h[dict(dataset='QCD', flavor=sum, eta=slice(1.4j,None,sum))] +  h[dict(dataset='QCD', flavor=sum, eta=slice(None,-1.4j,sum))]
cqcd_hist = h[dict(dataset='QCD', flavor=sum, eta=slice(-1.4j, 1.4j,sum))]
tt_b_hist = h[dict(dataset='ttbar', flavor=4j, eta=sum)]
tt_l_hist = h[dict(dataset='ttbar', flavor=0j, eta=sum)]

tab_data = read_hist(data_hist)
tab_fqcd = read_hist(fqcd_hist)
tab_cqcd = read_hist(cqcd_hist)
tab_tt_b = read_hist(tt_b_hist)
tab_tt_l = read_hist(tt_l_hist)

print(tab_data)

{'pt': array([(  0.,  25.), ( 25.,  50.), ( 50.,  75.), ( 75., 100.),
       (100., 125.), (125., 150.), (150., 175.), (175., 200.),
       (200., 225.), (225., 250.), (250., 275.), (275., 300.),
       (300., 325.), (325., 350.), (350., 375.), (375., 400.),
       (400., 425.), (425., 450.), (450., 475.), (475., 500.)],
      dtype=[('f0', '<f4'), ('f1', '<f4')]), 'hist_value': array([27405., 21382., 16585., 12740., 10069.,  7878.,  6007.,  4678.,
        3666.,  2903.,  2333.,  1734.,  1352.,  1048.,   851.,   634.,
         485.,   401.,   294.,   230.]), 'hist_variance': array([27405., 21382., 16585., 12740., 10069.,  7878.,  6007.,  4678.,
        3666.,  2903.,  2333.,  1734.,  1352.,  1048.,   851.,   634.,
         485.,   401.,   294.,   230.])}


All axes remaining will generate a corresponding array that can be used to
declare `Variable` instances. Notice that because the histogram was declared
with `storage=Weight`, entries for `hist_value` (sum of weights) and
`hist_variance` (sum of weight-squared) will be presented to the user. This
information can the be used for the uncertainty generation.

In [4]:
from hepdata_lib import Table, Variable, Uncertainty
import hist.intervals

tab1d = Table("my table")

var = Variable("Jet pT", is_independent=True, is_binned=True, units="GeV")
var.values = tab_data["pt"]
tab1d.add_variable(var)

# Filling in the data entries
data_var = Variable(
    "data", is_independent=False, is_binned=False, units="Number of jets"
)
data_var.values = tab_data["hist_value"]
tab1d.add_variable(data_var)

# Fill and example MC entry with uncertainty
fqcd_var = Variable(
    "QCD (MC) forward region",
    is_independent=False,
    is_binned=False,
    units="Number of jets",
)
fqcd_var.values = tab_fqcd["hist_value"]
fqcd_stat = Uncertainty("statistical uncertainty", is_symmetric=False)
s, s2 = tab_fqcd["hist_value"], tab_fqcd["hist_variance"]
lo, up = hist.intervals.poisson_interval(s, s2)
lo, up = lo - s, up - s
fqcd_stat.values = zip(lo, up)
fqcd_var.add_uncertainty(fqcd_stat)
tab1d.add_variable(fqcd_var)


To improve the readability of uncertainty variable declaration, the `hist_utils`
package also provides, `hist_as_variable` method to easily declare variables.
Uncertainties should be provided using a dictionary, with the dictionary keys
being the desired name of the uncertainty, and the dictionary values being how
the uncertainties should be defined. This can either be a string indicating how
the uncertainty should be calculated (`poisson_sym` or `poisson_asym`), a
floating point (pair) to indicate flat, a histogram (pair) of the same format as
the input histogram representing the uncertainty, or a numpy array (pair) the is
compatible with the final array output.

In [5]:
from hepdata_lib.hist_utils import hist_as_variable

tt_b_var = hist_as_variable(
    "ttbar b jets",
    tt_b_hist,
    uncertainty={
        "statistical": "poisson_sym",
        "flat symmetric uncertainty": 0.1,
        "flat asymmetric uncertainty": (-0.02, +0.03),
        "asymmetric uncertainty from histogram": (
            tt_l_hist * (-0.02),
            tt_l_hist * 0.03,
        ),
    },
)

tab1d.add_variable(tt_b_var)


## Example using N-dimensional histogram 

Notice that because the flexibility of the scikit-hep histogram syntax, the same
syntax can be used for the histograms of arbitrary dimensions


In [6]:
tab2d = Table("my table 2d")


data_hist = h[dict(dataset="data", flavor=sum)]
qcd_hist = h[dict(dataset="QCD", flavor=sum)]
tt_b_hist = h[dict(dataset="ttbar", flavor=4j)]
tt_l_hist = h[dict(dataset="ttbar", flavor=0j)]

tab_data = read_hist(data_hist)
# tab_qcd = read_hist(qcd_hist)
# tab_tt_b = read_hist(tt_b_hist)
# tab_tt_l = read_hist(tt_l_hist)

pt_var = Variable("Jet pT", is_independent=True, is_binned=True, units="GeV")
pt_var.values = tab_data["pt"]
tab2d.add_variable(pt_var)
eta_var = Variable("Jet eta", is_independent=True, is_binned=True)
eta_var.values = tab_data["eta"]
tab2d.add_variable(eta_var)

# Filling in the data entries
data_var = Variable(
    "data", is_independent=False, is_binned=False, units="Number of jets"
)
data_var.values = tab_data["hist_value"]
tab2d.add_variable(data_var)

## Fill QCD, just statistical uncertainty
qcd_var = hist_as_variable(
    "QCD MC", qcd_hist, uncertainty={"statistical": "poisson_asym"}
)
tab2d.add_variable(qcd_var)


To further simply the construction of table from N-dimensional histograms, we
provide a `create_hist_base_table` function such that the axes variables are
automatically setup.


In [7]:
from hepdata_lib.hist_utils import create_hist_base_table

tab2d_v2 = create_hist_base_table(
    "my simplifed 2d table",
    data_hist,
    axes_rename={"pt": "Jet pT", "eta": "Jet eta"},
    axes_units={"pt": "GeV"},
)

tab2d_v2.add_variable(hist_as_variable("Data", data_hist))
tab2d_v2.add_variable(
    hist_as_variable(
        "ttbar l jets", tt_l_hist, uncertainty={"statistical": "poisson_sym"}
    )
)


## Outputting the submission

Finally, we can add the table to the sumission and create the files. Please
refer to the [Getting started notebook](Getting_started.ipynb) for a complete
example.

In [9]:
submission.add_table(tab1d)
submission.add_table(tab2d)
submission.add_table(tab2d_v2)
submission.create_files("example_output_hist", remove_old=True)
